# Middleware Cloud Function User-Model

## Import Library

In [1]:
import numpy as np
import json
import base64
import googleapiclient.discovery

from PIL import Image
from io import BytesIO
from flask import current_app as app

## Inference Function

In [2]:
# dev
# from google.colab import auth
# auth.authenticate_user()

### Constans Model Location

In [3]:
PROJECT = 'cosmic-quarter-312712'
MODEL = 'waste'
LIST_LABELS = ['cardboard', 'glass', 'metal', 'paper', 'plastic']

### Payload Helper

In [4]:
def _generate_payload(images):
    return {"instances": images}

### Request Helper

In [5]:
def _get_model_prediction(service, project, model='demo_model',
                          version=None, body=None):
    if body is None:
        raise NotImplementedError(
            f"_get_model_prediction didn't get any payload for model {model}")

    url = f'projects/{project}/models/{model}'
    if version:
        url += f'/versions/{version}'

    response = service.projects().predict(name=url, body=body).execute()
    return response

In [6]:
def _connect_service():
    kwargs = {'serviceName': 'ml', 'version': 'v1'}
    return googleapiclient.discovery.build(**kwargs)

In [7]:
def _preprocessing_image(img_64_encode):
  img_64_decode = base64.b64decode(img_64_encode)
  img = Image.open(BytesIO(base64.b64decode(img_64_encode)))

  # resize
  size = (256,256)
  img = img.resize(size)
  img = np.array(img)
  xs = np.array([ img ])

  return xs.tolist()

In [8]:
def _map_labels(x, y):
  return {
      'name': x,
      'confident': y
  }

In [20]:
def get_inference_image(request):
    request_json = request.get_json(silent=True)
    # request_json = request # dev
    image = request_json['image'] # base64

    image = _preprocessing_image(image)

    service = _connect_service()
    project = PROJECT
    model = MODEL
    response = _get_model_prediction(service, project,
                                     model=model,
                                     body=_generate_payload(image))
    result = map(_map_labels, LIST_LABELS, response['predictions'][0]['dense'])
    return json.dumps(list(result))

## Testing

In [21]:
# img = open('cardboard1.jpg', 'rb')
# img_read = img.read()
# img_64_encode = base64.encodebytes(img_read).decode()
# print(json.dumps({ 'image': img_64_encode }))
# get_inference_image({ 'image': img_64_encode })

{"image": "...\nimB//9k=\n"}


'[{"name": "cardboard", "confident": 0.9994144439697266}, {"name": "glass", "confident": 0.00010214486246695742}, {"name": "metal", "confident": 0.00012465484905987978}, {"name": "paper", "confident": 0.0003468661743681878}, {"name": "plastic", "confident": 1.2077418432454579e-05}]'